In [1]:
import numpy as np
import tensorflow as tf

# reload model
model_status = tf.keras.models.load_model('model_status')
# "model_status" is a folder, and it should be the same place as the app.py
model_status.summary() # Check architecture, can be omitted when deployed

# reload the second model
model_months = tf.keras.models.load_model('model_months')
# "model_months" is a folder, and it should be the same place as the app.py
model_months.summary() # Check architecture, can be omitted when deployed

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 52)                208       
_________________________________________________________________
dense (Dense)                (None, 52)                2756      
_________________________________________________________________
batch_normalization_1 (Batch (None, 52)                208       
_________________________________________________________________
dense_1 (Dense)              (None, 208)               11024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 208)               832       
_________________________________________________________________
dense_2 (Dense)              (None, 156)               32604     
_________________________________________________________________
batch_normalization_3 (Batch (None, 156)               6

In [2]:
# example user input
age = -10000
gender = 1
education = 0 # OH
car = 1
realty = 1
housing = 5 # OH
family = 10
marital = 3 # OH (family status)
children = 0
income_type = 2 # OH
occupation = 9 # OH
income = 1000000
employ_age = -5000
mobile = 1
work_phone = 1
fixed_line = 1
email = 1

In [3]:
# encoding
input_encoded = np.zeros(52)
# 12 features [0:12]
input_encoded[0] = gender
input_encoded[1] = car
input_encoded[2] = realty
input_encoded[3] = children
input_encoded[4] = income
input_encoded[5] = age
input_encoded[6] = employ_age
input_encoded[7] = mobile
input_encoded[8] = work_phone
input_encoded[9] =  fixed_line
input_encoded[10] = email
input_encoded[11] = family
# income type [12:16]
input_encoded[12 + income_type] = 1
# housing type [17:22]
input_encoded[17 + housing] = 1
# education type [23:27]
input_encoded[23 + education] = 1
# family status [28:32]
input_encoded[28 + marital] = 1
# occupation type [33:51]
input_encoded[33 + occupation] = 1
print(input_encoded)

[ 1.e+00  1.e+00  1.e+00  0.e+00  1.e+06 -1.e+04 -5.e+03  1.e+00  1.e+00
  1.e+00  1.e+00  1.e+01  0.e+00  0.e+00  1.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  1.e+00  1.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  1.e+00  0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  1.e+00  0.e+00  0.e+00
  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]


### Input shape for classifier

These 12 features can be used directly.

    CODE_GENDER                1.0
    FLAG_OWN_CAR               1.0
    FLAG_OWN_REALTY             1.0
    CNT_CHILDREN               0.0
    AMT_INCOME_TOTAL            427500.0
    DAYS_BIRTH                -12005.0
    DAYS_EMPLOYED              -4542.0
    FLAG_MOBIL                1.0
    FLAG_WORK_PHONE             1.0
    FLAG_PHONE                0.0
    FLAG_EMAIL                0.0
    CNT_FAM_MEMBERS             2.0
These 5 features need one hot encoding.

    INCOME_TYPE(OH).0           0.0
    INCOME_TYPE(OH).1           0.0
    INCOME_TYPE(OH).2           0.0
    INCOME_TYPE(OH).3           0.0
    INCOME_TYPE(OH).4           1.0
        HOUSING_TYPE(OH).0            0.0
        HOUSING_TYPE(OH).1            0.0
        HOUSING_TYPE(OH).2            0.0
        HOUSING_TYPE(OH).3            0.0
        HOUSING_TYPE(OH).4            1.0
        HOUSING_TYPE(OH).5            0.0
            EDUCATION_TYPE(OH).0          0.0
            EDUCATION_TYPE(OH).1          1.0
            EDUCATION_TYPE(OH).2          0.0
            EDUCATION_TYPE(OH).3          0.0
            EDUCATION_TYPE(OH).4          0.0
                FAMILY_STATUS(OH).0            1.0
                FAMILY_STATUS(OH).1            0.0
                FAMILY_STATUS(OH).2            0.0
                FAMILY_STATUS(OH).3            0.0
                FAMILY_STATUS(OH).4            0.0
                    OCCUPATION_TYPE(OH).0          1.0
                    OCCUPATION_TYPE(OH).1          0.0
                    OCCUPATION_TYPE(OH).2          0.0
                    OCCUPATION_TYPE(OH).3          0.0
                    OCCUPATION_TYPE(OH).4          0.0
                    OCCUPATION_TYPE(OH).5          0.0
                    OCCUPATION_TYPE(OH).6          0.0
                    OCCUPATION_TYPE(OH).7          0.0
                    OCCUPATION_TYPE(OH).8          0.0
                    OCCUPATION_TYPE(OH).9          0.0
                    OCCUPATION_TYPE(OH).10         0.0
                    OCCUPATION_TYPE(OH).11         0.0
                    OCCUPATION_TYPE(OH).12         0.0
                    OCCUPATION_TYPE(OH).13         0.0
                    OCCUPATION_TYPE(OH).14         0.0
                    OCCUPATION_TYPE(OH).15         0.0
                    OCCUPATION_TYPE(OH).16         0.0
                    OCCUPATION_TYPE(OH).17         0.0
                    OCCUPATION_TYPE(OH).18         0.0

In [4]:
def sigmoid(x):
    y = 1/(1 + np.exp(-x))
    return y

In [5]:
def activation(y):
    """
    modded sigmoid function
    Input
    - y: (float) output of the classifier, shape 1X2
        [1, 0] => no overdue
        [0, 1] => potential overdue
    Return
    - pred: (int) prediction, scalar
        0 => potential credit card
        1 => no credit card
    - poss: (float) possibility of getting a credit card
    """
    if (y[0,0] >= y[0,1]):
        pred = 0; # give prediction
        poss = sigmoid(y[0, 0]*2);
    else:
        pred = 1; # give prediction
        poss = 1 - sigmoid(y[0, 1]*2);
    return (pred, poss)

### activation Example
Weight average is applied because prediction based on month is stricter.

In [6]:
P_status = activation(model_status.predict(input_encoded.reshape(1, 52)))[1]
P_months = activation(model_months.predict(input_encoded.reshape(1, 52)))[1]
P = P_status * 2/5 + P_months * 3/5 # weight average
print("test result: {:.4f}".format(P))

test result: 0.4238
